In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)  
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator
from gestures import GESTURES


import datetime

In [2]:
batch_size=64
epochs=30
input_shape = (20,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5',batchSize=batch_size, dim=input_shape)

In [3]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.3))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(64)(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(16, activation='relu')(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [4]:
model.fit(trainGen,epochs=epochs)

Epoch 1/30
54/54 [==============================] - 9s 8ms/step - loss: 2.2234 - accuracy: 0.1297
Epoch 2/30
54/54 [==============================] - 0s 8ms/step - loss: 1.7858 - accuracy: 0.4116
Epoch 3/30
54/54 [==============================] - 0s 8ms/step - loss: 1.3045 - accuracy: 0.5159
Epoch 4/30
54/54 [==============================] - 0s 8ms/step - loss: 0.9899 - accuracy: 0.6018
Epoch 5/30
54/54 [==============================] - 0s 8ms/step - loss: 0.8373 - accuracy: 0.6810
Epoch 6/30
54/54 [==============================] - 0s 9ms/step - loss: 0.6124 - accuracy: 0.7667
Epoch 7/30
54/54 [==============================] - 0s 8ms/step - loss: 0.5294 - accuracy: 0.7920
Epoch 8/30
54/54 [==============================] - 0s 8ms/step - loss: 0.4948 - accuracy: 0.8128
Epoch 9/30
54/54 [==============================] - 0s 8ms/step - loss: 0.4513 - accuracy: 0.8294
Epoch 10/30
54/54 [==============================] - 0s 8ms/step - loss: 0.3946 - accuracy: 0.8484
Epoch 11/30
54/54 [

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tfliteModel = converter.convert()
modelPath = "saved_models/MODEL-{}.tflite".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
with open(modelPath, 'wb') as f:
  f.write(tfliteModel)

INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmpxnghnlhf\assets
INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmpxnghnlhf\assets
